<a href="https://githubtocolab.com/giswqs/geemap/blob/master/examples/notebooks/43_extract_values_to_points.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open in Colab"/></a>

Uncomment the following line to install [geemap](https://geemap.org) if needed.

In [ ]:
import os
import ee
import geemap
import geopandas
Map = geemap.Map()
Map

In [ ]:
# FULL SCRIPT
import os
import ee
import geemap
import geopandas
import glob

# Define Variables
id_attributte = 'FieldId';
polygon_name = ee.String('users/batuhankavlak4/DOKTAR_SAMPLE/TAT_DATASET_2020_v12');
ee_dataset = "COPERNICUS/S2_SR";
date_start = ee.Date('2020-02-01');
date_end = ee.Date('2020-11-30');
cloud_perc = 20;
band_names = ['B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B11', 'B12'];
output_dir = os.path.expanduser('./csv/sentinel2');
output_prefix = 'S2_FIELDMEAN_F20200201T20201130'
done_id_list = list(map(lambda x : int(x.split('_')[-1].split('.')[0]),
                        glob.glob(output_dir + "/*.csv")))


def set_field_id(id_int):
    '''
    Adds integer as FieldId att to an EE feature object.

    Parameters:
        feature: an EE feature object.
        id_int (int): Id to be added.

    Returns:
        feature(ee feature): an EE feature object. 
    '''
    def wrapper_f(feature):
        base_dict = ee.Dictionary({
            'FieldId': id_int
        })
        feature = feature.set(base_dict)
        return feature
    return wrapper_f


def set_coord_att(feature):
    '''
    Adds coordinates as Coords att to an EE feature object.

    Parameters:
        feature: an EE feature object.

    Returns:
        feature(ee feature): an EE feature object. 
    '''
    coords = feature.geometry().coordinates()
    base_dict = ee.Dictionary({
        'Coords': coords
    })
    feature = feature.set(base_dict)
    return feature


# Generic Function to remove a property from a feature
def remove_property(property_name):
    '''
    Remove only one att from an EE feature object.

    Parameters:
        feature: an EE feature object.
        property_name (str): Attribute to be dropped.

    Returns:
        feature(ee feature): an EE feature object. 
    '''
    def wrapper_f(feature):
        properties = feature.propertyNames()
        select_properties = properties.filter(ee.Filter.neq('item', property_name))
        return feature.select(select_properties)
    return wrapper_f
        

# Function for iteraton over the range of dates
def day_mosaics(date, newlist):
    '''
    Mosaics images in a given date parameter.

    Parameters:
        date: an EE feature object.
        newlist (ee List): Empty ee.List.

    Returns:
        image(ee Image): an EE Image object. 
    '''
    # Cast
    date = ee.Date(date);
    newlist = ee.List(newlist);
    
    # Filter collection between date and the next day
    filtered = s2_col.filterDate(date, date.advance(ee.Number(1), 'day')); ### img_col should be parametrized with a wrapper
    
    # Make the mosaic
    image = ee.Image(filtered.mosaic().set('day', filtered.first().id()));
    
    # Add system:index, system:time_end, and PRODUCT_ID
    image = image.setMulti({'system:index': filtered.first().get('system:index'),
                            'system:time_start': filtered.first().get('system:time_start'),
                            'PRODUCT_ID': filtered.first().get('PRODUCT_ID')
                           });
    
    # Add the mosaic to a list only if the collection has images
    return ee.List(ee.Algorithms.If(filtered.size(), newlist.add(image), newlist));


def advance_day(day):
    inc_date = ee.String(date_start.advance(day, 'day').format('YYYY-MM-dd')) ### class_region should be parametrized with a wrapper
    return inc_date


# Get Target Polygon File
polygon_collection = ee.FeatureCollection(polygon_name);
# Convert  in a Python list
polygon_collection_list = polygon_collection.aggregate_array(id_attributte);
polygon_collection_list_py = polygon_collection_list.getInfo();

# LOOP 1 - OVER POLYGONS
for field_id in polygon_collection_list_py:

    # Skip loop if already queried
    if done_id_list.count(field_id) == 1:
        print('Already did ' + str(field_id))
        continue

    # Get target polygon
    polygon_feature = ee.Feature(polygon_collection.filter(ee.Filter.eq(id_attributte, field_id)).first());
    polygon_feature = ee.FeatureCollection(polygon_feature)

    # Add Earth Engine dataset
    s2_col = ee.ImageCollection(ee_dataset) \
    .filterBounds(polygon_feature.geometry()) \
    .filterDate(date_start, date_end) \
    .filter(ee.Filter.lte('CLOUDY_PIXEL_PERCENTAGE', cloud_perc)) \
    .select(band_names);

    # Difference in days between start and finish
    diff = date_end.difference(date_start, 'day');

    # Make a list of all dates
    day_range = ee.List.sequence(ee.Number(0), diff.subtract(ee.Number(1))).map(advance_day);

    # Iterate over the range to make a new list, and then cast the list to an imagecollection
    img_mosaic_col = ee.ImageCollection(ee.List(day_range.iterate(day_mosaics, ee.List([]))));

    # Prepare Exported File Name
    out_csv = os.path.join(output_dir, output_prefix + '_' + str(field_id) + '.csv');
    # Export Polygon Stat
    try:
        geemap.zonal_statistics(img_mosaic_col, polygon_feature, out_csv, statistics_type='MEAN', scale=10)
    except:
        print('I had an exception here: ' + str(field_id))